In [1]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Accessing as aryan0147

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [2]:
# Set or create an experiment
mlflow.set_experiment("ML-Algos")

<Experiment: artifact_location='mlflow-artifacts:/d43192cc72bb48deb03d07938721755b', creation_time=1739197271560, experiment_id='12', last_update_time=1739197271560, lifecycle_stage='active', name='ML-Algos', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [4]:
df = pd.read_csv('../reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [5]:
import mlflow
import optuna
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 2: TF-IDF vectorization setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

# **Step 3: Train-test split BEFORE any class balancing (Prevents Data Leakage)**
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# **Step 4: Fit TF-IDF only on training data**
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

# **Step 5: Compute Class Weights Instead of SMOTE**
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ClassWeights_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup with computed class weights
    model = RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth,
        min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
        class_weight=class_weight_dict, random_state=42
    )

    # Fit and evaluate the model
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        class_weight=class_weight_dict, random_state=42
    )

    # Log the best model with MLflow
    log_mlflow("RandomForest", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()


[I 2025-02-10 20:19:34,067] A new study created in memory with name: no-name-56edfdb8-a04c-48da-90ce-0a2f512d5bd6
[I 2025-02-10 20:19:37,706] Trial 0 finished with value: 0.6682121914632483 and parameters: {'n_estimators': 236, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.6682121914632483.
[I 2025-02-10 20:19:40,064] Trial 1 finished with value: 0.6417564434747034 and parameters: {'n_estimators': 277, 'max_depth': 6, 'min_samples_split': 14, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.6682121914632483.
[I 2025-02-10 20:19:42,147] Trial 2 finished with value: 0.6553934269739533 and parameters: {'n_estimators': 153, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.6682121914632483.
[I 2025-02-10 20:19:46,008] Trial 3 finished with value: 0.6628937678985408 and parameters: {'n_estimators': 256, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 0 with valu

🏃 View run RandomForest_ClassWeights_TFIDF_Trigrams at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12/runs/01df6f7464c444aea765e4240e60b5e7
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12
